### Output sensor locations

- Select all sensors in area of study
- Re-format in correct format for sumo / Dfrouter
- Output list/index of sensors ready for next code to generate time series

Format of detector file:

```
<detectors>
    <detectorDefinition id="<DETECTOR_ID>" lane="<LANE_ID>" pos="<POS>"/>
... further detectors ...
</detectors>
```

https://sumo.dlr.de/docs/Demand/Routes_from_Observation_Points.html#computing_detector_types

In [1]:
import osmnx as ox
import pandas as pd
from shapely.geometry import box, Polygon, MultiPolygon, Point, LineString, mapping
import geopandas as gpd
import geojson
import mysql.connector
import csv
from shapely.ops import split, snap
from sklearn.neighbors import NearestNeighbors
from math import radians, cos, sin, asin, sqrt
import numpy as np
import utm

def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    # Radius of earth in kilometers is 6371
    km = 6371* c
    return km

In [2]:
# Get sensors
sensors = pd.read_csv('Data/midas_sensor_locations.csv')

#Get sensor distances
distance_df = pd.read_csv('Data/sensors_with_distances.csv', index_col=0)

sensors = sensors.merge(distance_df[['site ID','sp length']],left_on = 'site_ID',right_on = 'site ID', how='left').drop('site ID', axis=1)

#Read in sensors for A20 route

with open('Data/route1_a20.csv', newline='') as f:
    a20_sensors = f.read().splitlines()
print(a20_sensors)

a20_sensor = sensors[sensors['site_ID'].isin(a20_sensors)]

a20_sensor = a20_sensor.sort_values('sp length')

['5892/1', '5889/1', 'A20/7113A', 'M20/7095A', 'M20/7076A', 'M20/7018A', 'M20/6913A', 'M20/6905A', 'M20/6868A', 'M20/6645A', 'M20/6590A', 'M20/6582A', 'M20/6576A', 'M20/6572A', 'M20/6570A', 'M20/6568A1', 'M20/6552A1', 'M20/6545A', 'M20/6547A1', 'M20/6540A', 'M20/6534A', 'M20/6538A', 'M20/6531A', 'M20/6526A', 'M20/6518A', 'M20/6520A', 'M20/6523A', 'M20/6514A', 'M20/6517A', 'M20/6506A', 'M20/6511A', 'M20/6501A', 'M20/6498A', 'M20/6481A', 'M20/6484A', 'M20/6487A', 'M20/6491A', 'M20/6494A', 'M20/6477A', 'M20/6460A', 'M20/6465A', 'M20/6468A', 'M20/6472A', 'M20/6476A', 'M20/6454A', 'M20/6404A']


In [3]:
#Define line object between all sensors

point1 = Point(a20_sensor.iloc[0]['Longitude'],a20_sensor.iloc[0]['Latitude'])

line_objects = []
buffered_line_objects = []
for i,r, in a20_sensor[1:].iterrows():
    point2 = Point(r['Longitude'],r['Latitude'])
    sensor_line = LineString([point1,point2])
    line_objects.append(sensor_line)
    buffered_line_objects.append(sensor_line.buffer(0.009999))
    point1 = point2

# Create a FeatureCollection from the list of LineString objects
feature_collection = geojson.FeatureCollection([geojson.Feature(geometry=line) for line in line_objects])
# Convert the FeatureCollection to a GeoJSON string
geojson_string = geojson.dumps(feature_collection, indent=2)
    
# Convert buffered LineString objects to GeoJSON format
features = []
for buffered_line in buffered_line_objects:
    feature = geojson.Feature(geometry=mapping(buffered_line))
    features.append(feature)

# Create a FeatureCollection from the list of buffered LineString objects
feature_collection = geojson.FeatureCollection(features)

# Convert the FeatureCollection to a GeoJSON string
geojson_string = geojson.dumps(feature_collection, indent=2)
    
#Construct a polygon from line buffers

# Use unary_union to join the buffered LineString objects into a single polygon
A20_buffered_object = MultiPolygon(buffered_line_objects).buffer(0)  # Buffering with 0 effectively removes any small artifacts

In [4]:
# Extract route from OSMNX from buffer
#Get network for area just for pimary roads etc
cf = '["highway"~"motorway|motorway_link|primary|trunk"]'
G = ox.graph_from_polygon(A20_buffered_object, network_type='drive', custom_filter=cf, simplify=True)

node_attributes, edge_attributes = ox.graph_to_gdfs(G, nodes=True)

#Tidy Network
#Where there is multiple road types select 1 (at random)
road_types = []
for i in list(edge_attributes['highway']):
    if isinstance(i, str): 
        road_types.append(i)
    else:
        road_types.append(i[0])
edge_attributes['highway'] = road_types

#For edges with missing speed impute using mode
road_type_speed_limit = {}
for rtype in list(edge_attributes['highway'].value_counts().index):
    road_type_speed_limit[rtype] = edge_attributes[edge_attributes['highway'] == rtype]['maxspeed'].mode().values[0]

#Where speed missing or multuple, take from dict
speed = []
for i,r in edge_attributes.iterrows():
    #String case
    if isinstance(r['maxspeed'], str):
        speed.append(int("".join(filter(str.isdigit, r['maxspeed']))))
    #List case
    else:
        speed.append(int("".join(filter(str.isdigit, road_type_speed_limit[r['highway']]))))

edge_attributes['speed'] = speed

Each sensor described by:

- ID
- Lane
- Position along road

Process:
- Get ID for sensor
- Check how many lanes from TS data
- Expand by numer of lanes
- Adapt logic to split edges to find nodes, but get number of metres along road
    - Get proportion along road and then use to divide length
- Output in prescribed format

In [5]:
#Set up cursor object to query time-series database
host = "localhost"
user = "chris"
password = "password"
database = "midas"

# Establish a database connection
connection = mysql.connector.connect(
    host=host,
    user=user,
    password=password,
    database=database
)

cursor = connection.cursor(buffered=True)

#Get Lanes per Sensor

# Get time series data for sensor
id_list = list(a20_sensor['site_ID'])
sql_query = "select distinct site_ID, lane from full_data where site_ID in {} and yr = '2022'".format(tuple(id_list))
cursor.execute(sql_query)
result = cursor.fetchall()
sensor_id_lanes = pd.DataFrame(result, columns=[desc[0] for desc in cursor.description])

print(sensor_id_lanes)

sensor_points = []
for i,r in a20_sensor.iterrows():
    sensor_points.append(Point(r['Longitude'],r['Latitude']))
    
#Compute nearest edges - needs to be recomputed each time, as structure of G fundamentally changes with each iteration
sensor_nearest_edges, distances = ox.distance.nearest_edges(G, list(a20_sensor['Longitude']), list(a20_sensor['Latitude']), interpolate=None, return_dist=True)

        site_ID                         lane
0     M20/6481A                        lane4
1     M20/6468A                        lane4
2     M20/6404A                        lane3
3     M20/6531A                        lane4
4    M20/6547A1                        lane2
..          ...                          ...
143   M20/6913A                        lane3
144   M20/6905A                        lane2
145   M20/6491A                        lane3
146   M20/6868A                        lane2
147      5892/1  allLanesCompleteCarriageway

[148 rows x 2 columns]


In [6]:
sensor_details_list = []

for sens_ind in range(len(sensor_points)):
    
    sensor_details_append = {}
    
    #Get point
    sensor_point = sensor_points[sens_ind]
    #Get ID
    sensor_id = id_list[sens_ind]
    sensor_details_append['ID'] = sensor_id
    #Append Popint
    sensor_details_append['geometry'] = sensor_points[sens_ind]
    #Get edge
    edge = G.edges[sensor_nearest_edges[sens_ind][0],sensor_nearest_edges[sens_ind][1],0]
    sensor_details_append['Edge ID'] = tuple([sensor_nearest_edges[sens_ind][0],sensor_nearest_edges[sens_ind][1]])
    sensor_details_append['Highway Type'] = edge['highway']
    #Edge Geometry
    edge_geom = edge['geometry']
    line_coords = list(edge_geom.coords)

    # Flatten the coordinates for use with NearestNeighbors
    X = [(x, y) for x, y in line_coords]
    # Fit a NearestNeighbors model
    nn_model = NearestNeighbors(n_neighbors=1, algorithm='ball_tree').fit(X)
    # Query the model with the coordinates of the Point
    query_result = nn_model.kneighbors([[sensor_point.x, sensor_point.y]])
    # Get the index of the nearest point on the LineString
    nearest_point_index = query_result[1][0][0]
    # Extract coordinates of the nearest point on the LineString
    nearest_coordinates = line_coords[nearest_point_index]

    # Split edge geometry at nearest point
    split_line = split(edge_geom, Point(nearest_coordinates))

    #Get new geometries
    segments = [feature for feature in split_line.geoms]
    gdf_segments = gpd.GeoDataFrame(list(range(len(segments))), geometry=segments)
    gdf_segments.columns = ['index', 'geometry']

    if len(gdf_segments) > 1:
        proportion = gdf_segments.iloc[0]['geometry'].length / (gdf_segments.iloc[0]['geometry'].length + gdf_segments.iloc[1]['geometry'].length)
        metres = edge['length'] * proportion
        sensor_details_append['dist_m'] = metres
    else:
        geom_coords_list = [(x, y) for x, y in gdf_segments.iloc[0]['geometry'].coords]
        
        distances = []
        for p in geom_coords_list:
            distances.append(haversine(p[0],p[1],sensor_point.x,sensor_point.y) * 1000)
        if distances[0] < distances[-1]:
            metres = distances[0]
            sensor_details_append['dist_m'] = metres
        else:
            metres = -distances[1]
            sensor_details_append['dist_m'] = metres
            
    sensor_details_list.append(sensor_details_append)

In [7]:
sensor_details = pd.DataFrame(sensor_details_list)

In [9]:
sensor_details.to_csv('Data/Networks/route1/sensor_details.csv')

### Next steps

- Get osm network and convert sumo format
- Map sumo edges to OSM edges
- Convert sensor_details IDs to Sumo format
- Output as json as appropriate

In [8]:
# Read in .net.xml file

import xml.etree.ElementTree as ET

tree = ET.parse('Data/Networks/route1/route1.net.xml')
root = tree.getroot()

roads_list = []
lanes_list = []

for child in root:
    if child.tag == 'edge':
        roads_list.append(child.attrib)
        for lane in child:
            lanes_dict = lane.attrib
            try:
                lanes_dict['road_type'] = child.attrib['type']
            except:
                lanes_dict['road_type'] = None
            lanes_list.append(lanes_dict)

In [11]:
roads = pd.DataFrame(roads_list)
lanes = pd.DataFrame(lanes_list)

In [12]:
def remove_last_two_chars(s):
    return s[:-2]

lanes['id_strip'] = lanes['id'].apply(remove_last_two_chars)

In [15]:
offset = [310981.68,5660999.41]

In [ ]:
# Process

# Convert all shapes points in lanes to lat/lon
# Associate each to a lane_id
# knn on sensors to lane points
# for nearest points get lane
# get distance to start and end point of shape
# use this to get proportion of road and then divide length by this amount
# output and observe

In [16]:
road_types = list(lanes['road_type'].value_counts().index)

In [18]:
network_points = {}
network_coords = {}
ind_to_lane = {}

for type in list(lanes['road_type'].value_counts().index):
    network_points[type.split(".")[1]] = []
    network_coords[type.split(".")[1]] = []
    ind_to_lane[type.split(".")[1]] = {}
    ind_to_lane[type.split(".")[1]][-1] = None

In [19]:
for i,r, in lanes.iterrows():
    if r['road_type'] is not None:
        for coord in r['shape'].split():
            easting = float(coord.split(',')[0]) + offset[0]
            northing = float(coord.split(',')[1]) + offset[1]

            lon = utm.to_latlon(easting,northing,zone_number=31,northern=True)[0]
            lat = utm.to_latlon(easting,northing,zone_number=31,northern=True)[1]

            network_points[r['road_type'].split(".")[1]].append(Point(lat,lon))
            network_coords[r['road_type'].split(".")[1]].append([lon,lat])
            ind_to_lane[r['road_type'].split(".")[1]][max(list(ind_to_lane[r['road_type'].split(".")[1]].keys())) + 1] = r['id'][:-2]

In [50]:

sensor_snapping_list = []

for sens_ind in range(len(sensor_points)):

    sensor = sensor_details.iloc[sens_ind]
    sensor_road_type = sensor['Highway Type']

    #Pull back roads in network for particular road type
    road_locations = np.array(network_coords[sensor_road_type])
    # Create a NearestNeighbors object with k=1
    knn = NearestNeighbors(n_neighbors=1).fit(road_locations)

    sensor_snapping_append = {}
    sensor_snapping_append['Sensor ID'] = id_list[sens_ind]

    # Find the index of the nearest neighbor
    nearest_neighbor_index = knn.kneighbors(np.array([sensor['geometry'].y,sensor['geometry'].x]).reshape(1, -1), return_distance=False)[0]

    retrive_lane = lanes[lanes['id_strip'] == ind_to_lane[sensor_road_type][nearest_neighbor_index[0]]]

    sensor_snapping_append['geometry'] = network_points[sensor_road_type][nearest_neighbor_index[0]]
    sensor_snapping_append['Num lanes on new network'] = len(retrive_lane)
    sensor_snapping_append['Num lanes on ts data'] = len(sensor_id_lanes[sensor_id_lanes['site_ID'] == id_list[sens_ind]])

    #Get start point of road
    lane_geometry = retrive_lane.iloc[0]['shape']
    start = lane_geometry.split()[0]
    easting = float(start.split(',')[0]) + offset[0]
    northing = float(start.split(',')[1]) + offset[1]
    lon_start = utm.to_latlon(easting,northing,zone_number=31,northern=True)[0]
    lat_start = utm.to_latlon(easting,northing,zone_number=31,northern=True)[1]

    #Get end point of road    
    end = lane_geometry.split()[-1]
    easting = float(end.split(',')[0]) + offset[0]
    northing = float(end.split(',')[1]) + offset[1]
    lon_end = utm.to_latlon(easting,northing,zone_number=31,northern=True)[0]
    lat_end = utm.to_latlon(easting,northing,zone_number=31,northern=True)[1]

    proportion_along_road = haversine(sensor.geometry.y,sensor.geometry.x,lon_start,lat_start) / haversine(sensor.geometry.y,sensor.geometry.x,lon_start,lat_start) + haversine(sensor.geometry.y,sensor.geometry.x,lon_end,lat_end)
    metres = float(retrive_lane.iloc[0]['length']) * proportion_along_road
    sensor_snapping_append['metres'] = metres
    sensor_snapping_list.append(sensor_snapping_append)
    sensor_snapping_append['dist to sensor'] = haversine(sensor_snapping_append['geometry'].x,sensor_snapping_append['geometry'].y,sensor['geometry'].x,sensor['geometry'].y)
sensor_snapping = pd.DataFrame(sensor_snapping_list)

In [57]:
sensor_snapping[sensor_snapping['dist to sensor'] <= 0.5]

,Sensor ID,geometry,Num lanes on new network,Num lanes on ts data,metres,dist to sensor
0,5892/1,POINT (1.3243760613374498 51.12541819458482),2,1,611.183254,0.102333
1,5889/1,POINT (1.2860689095983304 51.11003497535428),2,1,261.880611,0.007272
2,A20/7113A,POINT (1.1720810847154357 51.10493809969889),2,2,767.578468,0.227235
3,M20/7095A,POINT (1.159276604599264 51.0958481678629),2,2,834.493110,0.195852
4,M20/7076A,POINT (1.1254898168839762 51.09078285752044),2,2,1505.960007,0.430382
5,M20/7018A,POINT (1.0510283656221437 51.09587622895365),2,3,121.051483,0.043715
6,M20/6913A,POINT (0.9152482392134437 51.13516244257701),3,3,2206.717077,0.225554
7,M20/6905A,POINT (0.9068851195636333 51.140767987404026),3,3,1350.687489,0.257558
8,M20/6868A,POINT (0.8731265177778814 51.15831290303901),3,3,2441.600756,0.263916
9,M20/6645A,POINT (0.609464306421946 51.26701025445628),3,3,2128.359679,0.155369


In [58]:
sensors_gdf = gpd.GeoDataFrame(sensor_snapping[sensor_snapping['dist to sensor'] <= 0.5], crs="EPSG:4326", geometry=sensor_snapping[sensor_snapping['dist to sensor'] <= 0.5]['geometry'])
sensors_gdf.to_file("Data/Networks/route1/sensor_snapping_new_network.json", driver="GeoJSON")

Notes to self for when i pick this up:

- Mathching still imperfect - view "sensor_snapping_new_network.json" in qgis along with sensor location to see this - but it may be acceptable for initialy analysis
- Get distance between matched point and sensor points and remove any that are really far
- If time output converted netork as geojson and view in qgis as may account for some of the mismatching
- Otherwise:
    - Select a day/period of study e.g. 7 - 10 am on very normal weekday 
    - Start to output time-series in correct format
    - Run dfrouter and observe od matrix and desire lines

18/03/24

Things to try today:

- Converting sensors to CRS of new network
    - Visualise
    - Does this improve matching?
- Associate edges in new network to edges of old network and then use old matching